In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
import math

In [2]:
PCA = False

In [3]:
if PCA:
    train = pd.read_csv('FinalPCATrain.csv',index_col=0)
else:
    train = pd.read_csv('FinalTrain.csv',index_col=0)

# Without Double Regression

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [5]:
train.head()

,NumberOfCustomers,NumberOfSales,HasPromotions,NearestCompetitor,Region_AreaKM2,Mean_Dew_PointC,Mean_Sea_Level_PressurehPa,Mean_VisibilityKm,Precipitationmm,StandardMarket,...,February,April,May,June,September,October,December,Nothing,Fog,Snow
0,495,2.266354,0.0,0.003281,0.291715,0.400000,0.848485,0.354839,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,608,2.302695,0.0,0.003281,0.291715,0.400000,0.803030,0.419355,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,665,2.304995,0.0,0.003281,0.291715,0.400000,0.757576,0.354839,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,630,2.290064,0.0,0.003281,0.291715,0.314286,0.757576,0.483871,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,763,2.324482,1.0,0.003281,0.291715,0.342857,0.696970,0.709677,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
train.columns

Index(['NumberOfCustomers', 'NumberOfSales', 'HasPromotions',
       'NearestCompetitor', 'Region_AreaKM2', 'Mean_Dew_PointC',
       'Mean_Sea_Level_PressurehPa', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'December',
       'Nothing', 'Fog', 'Snow'],
      dtype='object')

In [7]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [8]:
NumberOfSales = test_data['NumberOfSales']

In [9]:
target = train_data['NumberOfSales']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=50)
m1.fit(variables, target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=True, random_state=1234, verbose=0, warm_start=False)

In [10]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)
test_data = test_data.drop(labels='NumberOfCustomers',axis=1)

In [12]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))

0.5862392192590752


In [13]:
print(str(mean_squared_error(NumberOfSales,yp)))

0.0009584461479297965


# Double Regression

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [15]:
if PCA:
    train = pd.read_csv('FinalPCATrain.csv',index_col=0)
else:
    train = pd.read_csv('FinalTrain.csv',index_col=0)
    
train.head()

,NumberOfCustomers,NumberOfSales,HasPromotions,NearestCompetitor,Region_AreaKM2,Mean_Dew_PointC,Mean_Sea_Level_PressurehPa,Mean_VisibilityKm,Precipitationmm,StandardMarket,...,February,April,May,June,September,October,December,Nothing,Fog,Snow
0,495,2.266354,0.0,0.003281,0.291715,0.400000,0.848485,0.354839,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,608,2.302695,0.0,0.003281,0.291715,0.400000,0.803030,0.419355,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,665,2.304995,0.0,0.003281,0.291715,0.400000,0.757576,0.354839,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,630,2.290064,0.0,0.003281,0.291715,0.314286,0.757576,0.483871,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,763,2.324482,1.0,0.003281,0.291715,0.342857,0.696970,0.709677,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## 50

In [16]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [17]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [18]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

rfr = RandomForestRegressor(random_state=1234) 
rfr.set_params(n_estimators=50)
rfr.fit(variables, target)


cust = rfr.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust

m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=50)
m1.fit(variables, target)
m1.oob_score_  

0.7755768584917917

In [19]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [20]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-1391.1552815284049
3.2248244130622252


## 50 reducing max depth

In [21]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [22]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [23]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

rfr = RandomForestRegressor(random_state=1234,max_depth=50) 
rfr.set_params(n_estimators=50)
rfr.fit(variables, target)


cust = rfr.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust

m1 = RandomForestRegressor(oob_score=True,random_state=1234,max_depth=50) 
m1.set_params(n_estimators=50)
m1.fit(variables, target)
m1.oob_score_  

0.7732149628382522

In [24]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [25]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-1527.631708500277
3.540961929721438


## 75

In [6]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [7]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [8]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

rfr = RandomForestRegressor(random_state=1234) 
rfr.set_params(n_estimators=75)
rfr.fit(variables, target)


cust = rfr.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust

m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=75)
m1.fit(variables, target)
m1.oob_score_  

0.775395117962749

In [9]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [11]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-0.367840578599403
0.0031685012078550333


## 100

In [10]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [11]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [12]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=100)
m1.fit(variables, target)

    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=100)
m2.fit(variables, target)
    
m2.oob_score_

In [16]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [17]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

## 125

In [91]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [92]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [93]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=125)
m1.fit(variables, target)
    
    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=125)
m2.fit(variables, target)
m2.oob_score_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=1,
           oob_score=True, random_state=1234, verbose=0, warm_start=False)

In [ ]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [94]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

## 150

In [5]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [6]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [7]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=150)
m1.fit(variables, target)
    
    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=150)
m2.fit(variables, target)
m2.oob_score_

0.7975070406543506

In [8]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [9]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-1377.020339913655
3.1920818696108864


In [11]:
yp

array([0.42927986, 0.22501983, 0.41657731, ..., 0.24087278, 0.53151844,
       0.44351347])

In [12]:
NumberOfSales

308671    2.242744
344782    2.235271
413557    2.233817
102151    2.301304
331232    2.178203
395986    2.234992
356933    2.259066
149549    2.180649
99185     2.210892
280138    2.297457
381147    2.262648
362603    2.266573
150488    2.201208
282569    2.241944
356738    2.227747
329637    2.248294
133485    2.134382
351973    2.219563
31323     2.223869
159151    2.293522
139585    2.248603
414339    2.282379
74751     2.394906
193551    2.212599
292208    2.197524
275860    2.241684
260794    2.167087
64293     2.250198
220447    2.220333
403812    2.278756
            ...   
367012    2.138159
72159     2.243537
23744     2.211378
419434    2.208658
422557    2.214434
294794    2.217973
263682    2.247674
92727     2.271732
234145    2.252573
425030    2.278371
21241     2.249633
121268    2.251553
354620    2.177797
214118    2.249590
123687    2.211087
290019    2.264305
99093     2.228537
307181    2.232810
2350      2.202631
10594     2.256059
267413    2.212503
402303    2.

# Trying to build a regressor model for each type of store

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [2]:
if PCA:
    train = pd.read_csv('FinalPCATrain.csv',index_col=0)
else:
    train = pd.read_csv('FinalTrain.csv',index_col=0)

In [3]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [4]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

var_1 = variables.loc[variables['HyperMarket'] == 1]
var_2 = variables.loc[variables['SuperMarket'] == 1]
var_3 = variables.loc[variables['StandardMarket'] == 1]
var_4 = variables.loc[variables['ShoppingCenter'] == 1]

tar_1 = train_data.loc[train_data['HyperMarket'] == 1]['NumberOfCustomers']
tar_2 = train_data.loc[train_data['SuperMarket'] == 1]['NumberOfCustomers']
tar_3 = train_data.loc[train_data['StandardMarket'] == 1]['NumberOfCustomers']
tar_4 = train_data.loc[train_data['ShoppingCenter'] == 1]['NumberOfCustomers']

In [5]:
c1 = RandomForestRegressor(random_state=1234) 
c2 = RandomForestRegressor(random_state=1234) 
c3 = RandomForestRegressor(random_state=1234) 
c4 = RandomForestRegressor(random_state=1234) 
c1.set_params(n_estimators=125)
c2.set_params(n_estimators=125)
c3.set_params(n_estimators=125)
c4.set_params(n_estimators=125)
c1.fit(var_1, tar_1)
c2.fit(var_2, tar_2)
c3.fit(var_3, tar_3)
c4.fit(var_4, tar_4)

cust1 = c1.predict(var_1)
cust2 = c2.predict(var_2)
cust3 = c3.predict(var_3)
cust4 = c4.predict(var_4)

var_1['NumberOfCustomers'] = cust1
var_2['NumberOfCustomers'] = cust2
var_3['NumberOfCustomers'] = cust3
var_4['NumberOfCustomers'] = cust4

tar_1 = train_data.loc[train_data['HyperMarket'] == 1]['NumberOfSales']
tar_2 = train_data.loc[train_data['SuperMarket'] == 1]['NumberOfSales']
tar_3 = train_data.loc[train_data['StandardMarket'] == 1]['NumberOfSales']
tar_4 = train_data.loc[train_data['ShoppingCenter'] == 1]['NumberOfSales']



s1 = RandomForestRegressor(oob_score=True,random_state=1234) 
s2 = RandomForestRegressor(oob_score=True,random_state=1234) 
s3 = RandomForestRegressor(oob_score=True,random_state=1234) 
s4 = RandomForestRegressor(oob_score=True,random_state=1234)
s1.set_params(n_estimators=125)
s2.set_params(n_estimators=125)
s3.set_params(n_estimators=125)
s4.set_params(n_estimators=125)
s1.fit(var_1, tar_1)
s2.fit(var_2, tar_2)
s3.fit(var_3, tar_3)
s4.fit(var_4, tar_4)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\sit

NameError: name 'S4' is not defined

In [7]:
test_1 =  test_data.loc[test_data['HyperMarket'] == 1]
test_2 =  test_data.loc[test_data['SuperMarket'] == 1]
test_3 =  test_data.loc[test_data['StandardMarket'] == 1]
test_4 =  test_data.loc[test_data['ShoppingCenter'] == 1]

In [8]:
sales1 = test_1['NumberOfSales']
sales2 = test_2['NumberOfSales']
sales3 = test_3['NumberOfSales']
sales4 = test_4['NumberOfSales']

In [9]:
test_1 = test_1.drop(labels='NumberOfSales',axis=1)
test_2 = test_2.drop(labels='NumberOfSales',axis=1)
test_3 = test_3.drop(labels='NumberOfSales',axis=1)
test_4 = test_4.drop(labels='NumberOfSales',axis=1)

In [10]:
yp = s1.predict(test_1)
print(str(r2_score(sales1,yp)))
print(str(mean_squared_error(sales1,yp)))
yp = s2.predict(test_2)
print(str(r2_score(sales2,yp)))
print(str(mean_squared_error(sales2,yp)))
yp = s3.predict(test_3)
print(str(r2_score(sales3,yp)))
print(str(mean_squared_error(sales3,yp)))
yp = s4.predict(test_4)
print(str(r2_score(sales4,yp)))
print(str(mean_squared_error(sales4,yp)))

0.6390843396667892
0.00098727583135878
0.6895962886483329
0.0006364810986711153
0.6660428748543659
0.0005042486842456487
0.9406875528177541
0.00012673677152988746
